# Tweet Sentiment NLP

This notebook covers sentiment analysis for a collection of 1.6 million tweets, sourced from this Kaggle [dataset](https://www.kaggle.com/kazanova/sentiment140). My goal is build a series of machine learning models that can predict document polarity reasonably accurately, with each attempt attacking the classification problem from a different natural language processing approach and level of sophistication. The below code can be split into three sections: generalized text preprocessing, single-layer model evaluation, and multi-layer model evaluation. 

## Generalized Text Preprocessing

### Basic Corpus Cleaning

For the generalized text preprocessing section, modifications made are only those that do not intentionally "lose" tweet information (i.e. no dropping words or ther destructive changes). This then leaves only formatting, standardization, and other methods of purely data cleaning. See the "general_preprocessing" function for a full list and order of all data preparation techniques used. A general summary of changes made is as follows: replacing urls, usernames, emoticons, and unrecognized characters with words; expanding contractions and common abbreviations; removing all non-alphanumeric characters; and truncating egregiously long sequences of repeated characters. This should then leave the corpus comprised solely of recognized English language words and numbers.

In [1]:
# check if modules are installed
from subprocess import Popen, PIPE

output = Popen("pip list | awk '{print $1}'", shell = True, stdout=PIPE).stdout.read().split()
packages = [x.decode('utf-8') for x in output][2:]
modules = ['bayesian-optimization', 'contractions', 'kaggle', 'nltk', 'pandas', 'seaborn', 'scikit-learn']
for nm in modules:
    if nm not in packages:
        ! pip install {nm}

In [2]:
# importing modules and setup
import contractions
import csv
import glob
import matplotlib.pyplot as plt
import nltk
import os
import pandas as pd
import re
import seaborn as sns
import zipfile
from bayes_opt import BayesianOptimization
from kaggle.api.kaggle_api_extended import KaggleApi
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedShuffleSplit

In [3]:
# downloading dataset
if not os.path.isfile('df_data.pkl'):
    api = KaggleApi()
    api.authenticate()
    dataset = 'kazanova/sentiment140'
    csv_name = 'training.1600000.processed.noemoticon.csv'
    try:
        os.remove(csv_name)
    except:
        pass
    api.dataset_download_file(dataset, file_name=csv_name, path='./')
    fn = glob.glob('train*.zip', recursive = True)[0]
    with zipfile.ZipFile(fn) as zip_file:
        for file in zip_file.namelist():
            if file == csv_name:
                zip_file.extract(csv_name)
    os.remove(fn)
else:
    print('Skipping because df_data.pkl exists.')

Skipping because df_data.pkl exists.


In [4]:
# importing data
if not os.path.isfile('df_data.pkl'):
    columns = ['target', 'text']
    df_data = pd.read_csv(csv_name, usecols = [0, 5], header = None, names = columns)
    print(f'df_data dimensions: {df_data.shape}')
    df_data.head()
else:
    print('Skipping because df_data.pkl exists.')

Skipping because df_data.pkl exists.


In [5]:
# importing emoticon descriptions
if not os.path.isfile('df_data.pkl'):
    df_repl = pd.read_csv('emoticon_descriptions.csv', header = 0, usecols =[0, 1], \
        names = ['emoticon', 'description'])
    print(f'df_repl dimensions: {df_repl.shape}')
    df_repl.head()
else:
    print('Skipping because df_data.pkl exists.')

Skipping because df_data.pkl exists.


In [6]:
# moving to dictionary
if not os.path.isfile('df_data.pkl'):
    dict_emot = {a:b for a, b in zip(df_repl.iloc[:, 0], df_repl.iloc[:, 1])}
else:
    print('Skipping because df_data.pkl exists.')

Skipping because df_data.pkl exists.


In [7]:
# adding html replacements
if not os.path.isfile('df_data.pkl'):
    dict_emot['&quot;'] = 'quote'
    dict_emot['&amp;'] = 'and'
    dict_emot['&lt;'] = 'less than'
    dict_emot['&gt;'] = 'greater than'
else:
    print('Skipping because df_data.pkl exists.')

Skipping because df_data.pkl exists.


In [8]:
# creating dictionary of common abbreviations
if not os.path.isfile('df_data.pkl'):
    df_repl = pd.read_csv('common_abbreviations.csv').applymap(lambda x: x.lower())
    dict_abbr = {a:b for a, b in zip(df_repl.iloc[:, 0], df_repl.iloc[:, 1])}
else:
    print('Skipping because df_data.pkl exists.')

Skipping because df_data.pkl exists.


In [9]:
# general preprocessing tweet body text
def general_preprocess(text):
    '''
    Returns a generally-applicable preprocessed version of the passed string.

        Parameters:
            text (str) : passed string
        
        Returns:
            mod_text (str) : preprocessed string
    '''

    # add leading and trailing whitespace
    mod_text = ' ' + text + ' '

    # replace usernames
    mod_text = re.sub(r'(?<=\s)(@\S+)(?=\s)', ' USER ', mod_text)
    
    # replace urls
    mod_text = re.sub(r'(?<=\s)(https?:\/\/\S+)(?=\s)', ' URL ', mod_text)
    
    # replace non-space whitespace
    mod_text = re.sub(r'\s+', ' ', mod_text)
    
    # replace emoticons with text
    for i, k in dict_emot.items():
        mod_text = mod_text.replace(i, f' {k} ')
    
    # replace unrecognized characters
    mod_text = mod_text.replace('İ', 'I')
    
    # expand contractions
    mod_text = contractions.fix(mod_text)

    # remove non-alphanumeric characters
    mod_text = re.sub(r'[^a-zA-Z0-9]', ' ', mod_text)

    # lower case text
    mod_text = mod_text.lower()

    # truncate repeated characters
    mod_text = re.sub(r'(.)\1{2,}', r'\1\1', mod_text)

    # replace common abbreviations
    for i, k in dict_abbr.items():
        mod_text = mod_text.replace(f' {i} ', f' {k} ')
    
    # remove repeated spaces
    mod_text = re.sub(r'( )\1+', ' ', mod_text)

    # trim leading / trailing whitespace
    mod_text = mod_text.strip()

    return mod_text


In [10]:
# apply function to dataframe
if not os.path.isfile('df_data.pkl'):
    df_data.loc[:, 'text'] = df_data['text'].apply(general_preprocess)
    df_data.head()
else:
    print('Skipping because df_data.pkl exists.')

Skipping because df_data.pkl exists.


In [11]:
# doing some pickling
if not os.path.isfile('df_data.pkl'):
    df_data.to_pickle('df_data.pkl')
else:
    print('Skipping because df_data.pkl exists.')

Skipping because df_data.pkl exists.


## Single Layer Machine Learning Models

### More Processing

The most basic version of natural language encoding identifies documents by unique sets of word tokens and counts. The ML models in this section will all be trained using an optimized variant of this method, term frequency-inverse document frequency vectors. TF-IDF weights the word token counts to highlight words that are more important to a document, while vectorization converts the string of text into a more machine-comprehensible vector of numbers.

Two constraints will be used to futher filter and simplify our corpus dictionary: removing stopwords, and lemmatization. Both techniques will result in a loss of contextual information, hence their absence from the prior section, though this is a necessary tradeoff to keep our vector space small.

In [12]:
# downloading stopwords and lemmas from nltk
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
nltk.download('wordnet')
lemma = WordNetLemmatizer()

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/parkernisbet/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/parkernisbet/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [13]:
# additional preprocessing for machine learning methods
def sl_preprocess(text):
    '''
    Returns a preprocessed version of the passed string useful for single layer methods.
    
        Parameters:
            text (str) : passed string
        
        Returns:
            mod_text (str) : preprocessed string
    '''
    
    # split string into tokens, removing stopwords and single characters, lemmatize
    mod_text = ' '.join([lemma.lemmatize(i) for i in text.split() if len(i) > 1 if i not in stop_words])

    return mod_text

In [14]:
# applying function to dataframe
if not os.path.isfile('df_data_sl.pkl'):
    df_data.loc[:, 'text'] = df_data['text'].apply(sl_preprocess)
    df_data.head()
else:
    print('Skipping because df_data_sl.pkl exists.')

Skipping because df_data_sl.pkl exists.


In [15]:
# saving to pickle
if not os.path.isfile('df_data_sl.pkl'):
    df_data.to_pickle('df_data_sl.pkl')
else:
    df_data = pd.read_pickle('df_data_sl.pkl')
    print("df_data_sl.pkl read into df_data.")

df_data_sl.pkl read into df_data.


In [16]:
# test train split
sss = StratifiedShuffleSplit(n_splits = 1, test_size = .2, random_state = 3)
sss.get_n_splits(df_data.text, df_data.target)
for train_ind, test_ind in sss.split(df_data.text, df_data.target):
    pass
print(f'Train_ind shape: {train_ind.shape}\nTest_ind shape: {test_ind.shape}')


Train_ind shape: (1280000,)
Test_ind shape: (320000,)


In [17]:
# tfidf vectorizing
vectorizer = TfidfVectorizer(analyzer = 'word', max_features = 300000)
X_train = vectorizer.fit_transform(df_data.text[train_ind])
y_train = df_data.target[train_ind]
X_test = vectorizer.transform(df_data.text[test_ind])
y_test = df_data.target[test_ind]

### Model Evaluation

The vectorizer object above converts the dataframe of strings into a sparse matrix, so this slightly limits which sklearn models we can use for prediction. A detailed list of compatible methods can be found [here](https://dziganto.github.io/Sparse-Matrices-For-Efficient-Machine-Learning/), however this notebook will only cover a small subset of sklearn's classifiers:

    - LogisticRegression
    - SVC
    - MultinomialNB
    - DecisionTreeClassifier
    - RandomForestClassifier
    - AdaBoostClassifier
    - MLPClassifier

While the order the above classifiers are solved in does not matter, it is worth noting that the last three (RandomForestClassifier and down) are ensemble classifiers.

In [18]:
# dictionary for storing model performance
model_perf = {}

In [19]:
# logistic regression function for hyperparameter tuning
def log_func(C = 1, do_more = 0):
    '''
    Function to optimize via Bayesian optimization.and

        Parameters:
            C (float) : inverse regularization strength
            do_more (integer) : flag for function to store model performance information in model_perf
        
        Returns:
            acc (float) : prediction accuracy
    '''

    global model_perf
    model = LogisticRegression(C = C, max_iter = 1000, random_state = 4).fit(X_train, y_train)
    y_pred = model.predict(X_test)
    acc = 100*np.sum(y_pred == y_test)/y_test.size
    if do_more == 1:
        c_mat = confusion_matrix(y_test, t_pred)
        f1 = f1_score(y_test, y_pred)
        model_perf['log'] = (acc, c_mat, f1)
    return acc


In [ ]:
# finding our optimal logistic regression C value
bounds = {'C': (.001, 1000)}
opt = BayesianOptimization(f = log_func, pbounds = bounds, random_state = 4)
opt.maximize(init_points = 10, n_iter = 20)
print(opt.max)

In [ ]:
# storing the model perf
C = optimizer.max['params']['C']
print(f'Logistic Regression accuracy: {round(log_func(C = C, do_more = 1), 2)}')